In [ ]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from urllib.parse import urlparse
import secrets

load_dotenv()

def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()


def github_to_og(url: str) -> str:
    """
    将 GitHub 仓库链接转换为 OpenGraph 图片链接。
    例如: https://github.com/delaford/game
    ->   https://opengraph.githubassets.com/<random>/delaford/game
    """
    parsed = urlparse(url.strip())
    host = (parsed.netloc or "").lower()

    # 允许带 www
    if host.startswith("www."):
        host = host[4:]

    if host != "github.com":
        raise ValueError("仅支持 github.com 域名的仓库链接")

    parts = [p for p in parsed.path.split("/") if p]
    if len(parts) < 2:
        raise ValueError("链接路径不完整，应为 https://github.com/<owner>/<repo>")

    owner, repo = parts[0], parts[1]
    if repo.endswith(".git"):
        repo = repo[:-4]

    # 生成任意随机字符串（URL 安全）
    token = secrets.token_urlsafe(16)

    return f"https://opengraph.githubassets.com/{token}/{owner}/{repo}"

### 第一轮对话

In [15]:
#user_input = "https://github.com/delaford/game"
user_input = "帮我看一下这个网址 https://github.com/delaford/game"

system_prompt = """
请提取用户输入的网址，请按照以下 JSON 格式定义的格式输出结果：
<OUTPUT JSON SCHEMA>
```json
{
    "website": "提取到的用户输入的网址",
}
```
</OUTPUT JSON SCHEMA>
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_input}
]

llm = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url="https://api.openai.com/v1"
    )

llm_response = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=False
    )

user_input_url = json.loads(clean_json_tags(llm_response.choices[0].message.content))["website"]

### 第二轮对话

In [22]:
user_input2 = f"{user_input_url}"

system_prompt2 = """
根据用户所给的网址，1. 总结一下这个网址的内容，用一段话总结，不要bullet point。2. 根据总结的内容起一个自媒体的标题
请按照以下 JSON 格式定义的格式输出结果：
<OUTPUT JSON SCHEMA>
```json
{
	"conclusion": "非bullet point的形式总结的网址内容的一段话的总结"
	"title": "根据总结的内容起一个自媒体的标题"
}
```
</OUTPUT JSON SCHEMA>
"""

messages2 = [
    {"role": "system", "content": system_prompt2},
    {"role": "user", "content": user_input2}
]

llm_response = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages2,
    stream=False
    )

output_json = json.loads(clean_json_tags(llm_response.choices[0].message.content))

In [23]:
tutorial_url = user_input_url
image_url = github_to_og(tutorial_url)
conclusion = output_json["conclusion"]
title = output_json["title"]

In [25]:
markdown_template = f"""
# 🧑‍💻 {title}
### 🕒 时间与会议号
- **日期**：2025年8月11日（星期一）
- **时间**：晚上 **9:00 - 10:00**
- **时长**：约 1 小时
- **腾讯会议号**：600-5611-6847（长期）
---
### 📌 项目内容
![]({image_url}) 
---
### 📚 项目背景
- {conclusion}
---
### 🙋‍♂️ 如果你想提前了解
- 学习地址： {tutorial_url}
"""

with open('output.md', 'w', encoding='utf-8') as f:
    f.write(markdown_template)
